In [ ]:
from datetime import datetime, timedelta , time
import pytz
from collections import defaultdict

entered_priorities = []

class Task:
    def __init__(self):
        self.description = "empty"
        self.activity = "empty"
        self.time_required = 0.0
        self.time_unit = ""
        self.due_date = "2099-12-31"
        self.work_date = "2099-12-31"
        self.priority = 11
        self.start_time = "00:00"
        self.end_time = "00:00"
        self.dependencies = []

    def copy(self):
        new_task = Task()
        new_task.description = self.description
        new_task.activity = self.activity
        new_task.time_required = self.time_required
        new_task.time_unit = self.time_unit
        new_task.due_date = self.due_date
        new_task.work_date = self.work_date
        new_task.priority = self.priority
        new_task.start_time = self.start_time
        new_task.end_time = self.end_time
        new_task.dependencies = self.dependencies.copy()
        return new_task

    def get_description(self): return self.description
    def get_activity(self): return self.activity
    def get_time_required(self): return self.time_required
    def get_priority(self): return self.priority
    def get_due_date(self): return self.due_date
    def get_work_date(self): return self.work_date
    def get_start_time(self): return self.start_time
    def get_end_time(self): return self.end_time
    def get_dependencies(self): return self.dependencies

    def set_description(self, description): self.description = description
    def set_activity(self, activity): self.activity = activity
    def set_time_required(self, time_required): self.time_required = time_required
    def set_due_date(self, due_date): self.due_date = due_date
    def set_work_date(self, work_date): self.work_date = work_date
    def set_priority(self, priority): self.priority = priority
    def set_start_time(self, start_time): self.start_time = start_time
    def set_end_time(self, end_time): self.end_time = end_time
    def set_dependencies(self, dependencies): self.dependencies = dependencies

    def input_task(self, task_list):
        india_tz = pytz.timezone("Asia/Kolkata")
        now_ist = datetime.now(india_tz)

        self.description = input("📝 Enter task description: ")
        if not self.description.strip():
            print("Task description cannot be empty.")
            return False

        self.activity = input("🎯 Enter task activity: ")

        while True:
            print("\n⏱ Choose time unit:")
            print("1. Hours ⏳")
            print("2. Minutes ⌛")
            try:
                choice = int(input("Enter the number of the time unit (1 or 2.): "))
                if choice == 1:
                    self.time_required = float(input("Enter time required in hours: "))
                    self.time_unit = "hours"
                    break
                elif choice == 2:
                    self.time_required = float(input("Enter time required in minutes: ")) / 60
                    self.time_unit = "minutes"
                    break
                else:
                    print("❌ Invalid choice. Please select 1 or 2.")
            except ValueError:
                print("🤔 Invalid input. Please enter a valid number for time unit.")

        while True:
            due = input("📅 Enter due date (YYYY-MM-DD): ")
            try:
                if due.isdigit() and 1 <= int(due) <= 31:
                    day = int(due)
                    year = now_ist.year
                    month = now_ist.month
                    due_datetime = india_tz.localize(datetime(year, month, day, 23, 59, 59))
                else:

                     parsed_date = datetime.strptime(due, "%Y-%m-%d")
                     due_datetime = india_tz.localize(datetime.combine(parsed_date.date(), time(23, 59, 59)))

                self.due_date = due_datetime.strftime("%Y-%m-%d")

                if due_datetime.date() < now_ist.date():
                    delta = now_ist - due_datetime
                    print("⚠ Due date already passed! 😱")
                    print(f"⏳ Overdue by: {delta.days} day(s) 😡")
                    continue
                elif due_datetime.date() == now_ist.date():
                    midnight = india_tz.localize(datetime.combine(now_ist.date() + timedelta(days=1), datetime.min.time()))
                    delta = midnight - now_ist
                    h, rem = divmod(delta.seconds, 3600)
                    m, s = divmod(rem, 60)
                    print("🎉 Due date is today! 🥳")
                    print(f"⏳ Time left until midnight: {h} hour(s): {m} minute(s): {s} second(s).")
                else:
                    delta = due_datetime - now_ist
                    d = delta.days
                    h, rem = divmod(delta.seconds, 3600)
                    m, s = divmod(rem, 60)
                    print("📅 Due date is in the future. 😊")
                    print(f"⏳ Time left: {d} day(s): {h} hour(s): {m} minute(s): {s} second(s).")
                break
            except ValueError:
                print("🤯 Invalid date format. Please use %Y-%m-%d.")

        while True:
            work = input("📅 Enter work date (YYYY-MM-DD): ")
            try:
                work_datetime = india_tz.localize(datetime.strptime(work, "%Y-%m-%d"))
                due_date_obj = india_tz.localize(datetime.strptime(self.due_date, "%Y-%m-%d"))

                if work_datetime.date() < now_ist.date():
                    delta = now_ist - work_datetime
                    print("⚠ Work date already passed! 😱")
                    print(f"⏳ Overdue by: {delta.days} day(s) 😡")
                    continue
                elif work_datetime.date() > due_date_obj.date():
                    print("❌ Work date cannot be after the due date!")
                    print("📅 Please enter a work date on or before the due date.")
                    continue
                else:
                    self.work_date = work
                    break
            except ValueError:
                print("🤯 Invalid date format. Please use %Y-%m-%d.")

        while True:
            start = input("⏰ Enter start time (HH:MM) in 24 hrs: ")
            try:
                start_time_obj = datetime.strptime(start, "%H:%M").time()

                work_date_obj = datetime.strptime(self.work_date, "%Y-%m-%d").date()
                if work_date_obj == now_ist.date() and start_time_obj < now_ist.time():
                    print("❌ Start time is in the past for today's work date.")
                    continue

                self.start_time = start
                break
            except ValueError:
                print("❌ Invalid time format. Please use HH:MM.")

        while True:
            end = input("🏁 Enter end time (HH:MM) in 24 hrs: ")
            try:
                end_time_obj = datetime.strptime(end, "%H:%M").time()
                start_time_obj = datetime.strptime(self.start_time, "%H:%M").time()

                if end_time_obj <= start_time_obj:
                    print("❌ End time must be after start time.")
                    continue

                work_date_obj = datetime.strptime(self.work_date, "%Y-%m-%d").date()
                if work_date_obj == now_ist.date() and end_time_obj < now_ist.time():
                    print("❌ End time is in the past for today's work date.")
                    continue

                time_difference = datetime.combine(datetime.min, end_time_obj) - datetime.combine(datetime.min, start_time_obj)
                hours_difference = time_difference.seconds / 3600.0

                if hours_difference != self.time_required:
                    print(f"⚠ The time duration ({hours_difference} hours) does not match the time required ({self.time_required} {self.time_unit}).")
                    while True:
                        new_start = input("⏰ Enter new start time (HH:MM): ")
                        new_end = input("🏁 Enter new end time (HH:MM): ")
                        try:
                            new_start_time = datetime.strptime(new_start, "%H:%M").time()
                            new_end_time = datetime.strptime(new_end, "%H:%M").time()

                            if new_end_time <= new_start_time:
                                print("❌ End time must be after start time.")
                                continue

                            if work_date_obj == now_ist.date() and (new_start_time < now_ist.time() or new_end_time < now_ist.time()):
                                print("❌ Cannot set time in the past for today's work date.")
                                continue

                            new_time_diff = datetime.combine(datetime.min, new_end_time) - datetime.combine(datetime.min, new_start_time)
                            new_hours_diff = new_time_diff.seconds / 3600.0

                            if new_hours_diff == self.time_required:
                                self.start_time = new_start
                                self.end_time = new_end
                                break
                            else:
                                print(f"⚠ The time duration ({new_hours_diff} hours) does not match the time required ({self.time_required} {self.time_unit}).")
                        except ValueError:
                            print("❌ Invalid time format. Please use HH:MM.")
                    break
                else:
                    self.end_time = end
                    break
            except ValueError:
                print("❌ Invalid time format. Please use HH:MM.")

        # Input dependencies
        if task_list.number_of_tasks > 0:
            print("\n📋 Existing tasks (potential dependencies):")
            for i, t in enumerate(task_list.tasks, 1):
                print(f"{i}. {t.get_description()} (Activity: {t.get_activity()})")

            print("\n🔗 Enter task numbers this task depends on (comma separated, or leave empty if none):")
            dep_input = input("Dependencies: ").strip()
            if dep_input:
                dep_indices = [int(x.strip()) - 1 for x in dep_input.split(",") if x.strip().isdigit()]
                self.dependencies = [task_list.tasks[i].get_description() for i in dep_indices if 0 <= i < task_list.number_of_tasks]
                if self.dependencies:
                    if self.check_circular_dependencies(task_list, self.description, self.dependencies):
                        print("❌ Circular dependencies detected! Dependencies not set.")
                        self.dependencies = []
                    else:
                        print(f"✅ This task depends on: {', '.join(self.dependencies)}")
                else:
                    print("ℹ No valid dependencies selected.")

        if entered_priorities:
            print("\n📜 Previously entered priorities:")
            for i, (desc, prio) in enumerate(entered_priorities, start=1):
                print(f"{i}. {desc} (Priority: {prio})")

        while True:
            try:
                priority = int(input("⭐ Enter priority (1 = highest, 10 = lowest): "))
                if 1 <= priority <= 10:
                    existing_tasks = [t for t in task_list.tasks if t.get_priority() == priority]
                    if existing_tasks:
                        print(f"⚠ Warning: Priority {priority} is already assigned to task: '{existing_tasks[0].get_description()}'")
                        print("1. Change priority of the existing task")
                        print("2. Change priority of this new task")
                        choice = input("Which priority would you like to change? (1/2): ")
                        if choice == '1':
                            while True:
                                try:
                                    new_prio = int(input(f"Enter new priority for '{existing_tasks[0].get_description()}': "))
                                    if 1 <= new_prio <= 10:
                                        for i, (desc, prio) in enumerate(entered_priorities):
                                            if prio == priority and desc == existing_tasks[0].get_description():
                                                entered_priorities[i] = (desc, new_prio)
                                        existing_tasks[0].set_priority(new_prio)
                                        self.priority = priority
                                        entered_priorities.append((self.description, priority))
                                        break
                                    else:
                                        print("❌ Priority must be between 1 and 10.")
                                except ValueError:
                                    print("❌ Please enter a valid integer for priority.")
                        elif choice == '2':
                            continue
                        else:
                            print("❌ Invalid choice. Please try again.")
                            continue
                    else:
                        self.priority = priority
                        entered_priorities.append((self.description, priority))
                    break
                else:
                    print("❌ Priority must be between 1 and 10.")
            except ValueError:
                print("❌ Please enter a valid integer for priority.")
        return True

    def print_task(self):
        print("\n✨ Task Details ✨")
        print(f"📝 Description     : {self.description}")
        print(f"🎯 Activity        : {self.activity}")

        try:
            start_datetime = datetime.strptime(self.start_time, "%H:%M")
            end_datetime = datetime.strptime(self.end_time, "%H:%M")
            time_difference = end_datetime - start_datetime
            hours = time_difference.seconds // 3600
            minutes = (time_difference.seconds % 3600) // 60
            print(f"⏳ Duration        : {hours} hour(s) {minutes} minute(s)")
        except ValueError:
            print(f"⏳ Duration        : {self.time_required} hrs")

        print(f"📅 Due Date        : {self.due_date}")
        print(f"📅 Work Date       : {self.work_date}")
        print(f"⭐ User Priority   : {self.priority}")
        print(f"⏰ Start Time      : {self.start_time}")
        print(f"🏁 End Time        : {self.end_time}")
        if self.dependencies:
            print(f"🔗 Dependencies    : {', '.join(self.dependencies)}")
        else:
            print("🔗 Dependencies    : None")
        print("")

    def check_circular_dependencies(self, task_list, current_task_desc, dependencies, visited=None):
        if visited is None:
            visited = set()
        visited.add(current_task_desc)

        for dep_desc in dependencies:
            if dep_desc in visited:
                return True
            dep_task = next((t for t in task_list.tasks if t.get_description() == dep_desc), None)
            if dep_task:
                if self.check_circular_dependencies(task_list, dep_desc, dep_task.get_dependencies(), visited):
                    return True
        visited.remove(current_task_desc)
        return False

class TaskList:
    def __init__(self, max_tasks, name):
        self.name = name
        self.max_tasks = max_tasks
        self.number_of_tasks = 0
        self.tasks = []
        self.history_stack = []

    def get_name(self): return self.name

    def add_task(self, task):
        new_task = task.copy()

        for existing_task in self.tasks:
            if new_task.get_work_date() == existing_task.get_work_date():
                new_start = datetime.strptime(new_task.get_start_time(), "%H:%M").time()
                new_end = datetime.strptime(new_task.get_end_time(), "%H:%M").time()
                existing_start = datetime.strptime(existing_task.get_start_time(), "%H:%M").time()
                existing_end = datetime.strptime(existing_task.get_end_time(), "%H:%M").time()

                if (new_start < existing_end and new_end > existing_start):
                    print(f"⚠ Error: Time conflict with task: '{existing_task.get_description()}' 😤")
                    print(f"⏰ Existing task time: {existing_task.get_start_time()} to {existing_task.get_end_time()}")
                    print(f"📅 Work date: {existing_task.get_work_date()}")
                    print(f"🎯 Activity: {existing_task.get_activity()}")
                    while True:
                        while True:
                            new_task.set_start_time(input("⏰ Enter new start time (HH:MM): "))
                            new_task.set_end_time(input("🏁 Enter new end time (HH:MM): "))
                            try:
                                new_start = datetime.strptime(new_task.get_start_time(), "%H:%M").time()
                                new_end = datetime.strptime(new_task.get_end_time(), "%H:%M").time()
                                break
                            except ValueError:
                                print("❌ Invalid time format. Please use HH:MM.")

                        if not (new_start < existing_end and new_end > existing_start):
                            break
                        else:
                            print("⚠ Still a conflict. 😕")

        self.tasks.append(new_task)
        self.number_of_tasks += 1
        self.shift_up(self.number_of_tasks - 1)
        self.history_stack.append(('add', new_task.copy()))

    def remove_task(self):
        if self.number_of_tasks == 0:
            print("🚫 No tasks to remove. 🤷")
            return Task()

        for i in range(self.number_of_tasks):
            task = self.tasks[i]
            if not any(t.get_dependencies() for t in self.tasks if task.get_description() in t.get_dependencies()):
                removed = self.tasks[i]
                self.tasks[i] = self.tasks[self.number_of_tasks - 1]
                self.tasks.pop()
                self.number_of_tasks -= 1
                self.shift_down(i)
                self.history_stack.append(('remove', removed.copy()))
                return removed

        print("⚠ All tasks have dependencies that need to be completed first!")
        return Task()

    def undo_last_action(self):
        if not self.history_stack:
            print("↩ Nothing to undo. 😕")
            return
        action, task = self.history_stack.pop()
        if action == 'add':
            for i in range(self.number_of_tasks):
                if (self.tasks[i].get_description() == task.get_description() and
                    self.tasks[i].get_activity() == task.get_activity() and
                    self.tasks[i].get_priority() == task.get_priority() and
                    self.tasks[i].get_due_date() == task.get_due_date() and
                    self.tasks[i].get_start_time() == task.get_start_time() and
                    self.tasks[i].get_end_time() == task.get_end_time()):
                    self.tasks.pop(i)
                    self.number_of_tasks -= 1
                    print("✅ Undo successful: last added task removed. 🎉")
                    break
        elif action == 'remove':
            if self.number_of_tasks < self.max_tasks:
                self.tasks.append(task)
                self.number_of_tasks += 1
                self.shift_up(self.number_of_tasks - 1)
                print("✅ Undo successful: last removed task restored. 🥳")
            else:
                print("⚠ Cannot undo remove — task list is full. 😬")

    def shift_up(self, position):
        while position > 0 and self.tasks[self.parent(position)].get_priority() > self.tasks[position].get_priority():
            self.tasks[position], self.tasks[self.parent(position)] = self.tasks[self.parent(position)], self.tasks[position]
            position = self.parent(position)

    def shift_down(self, position):
        min_index = position
        l = self.left_child_node(position)
        r = self.right_child_node(position)
        if l < self.number_of_tasks and self.tasks[l].get_priority() < self.tasks[min_index].get_priority():
            min_index = l
        if r < self.number_of_tasks and self.tasks[r].get_priority() < self.tasks[min_index].get_priority():
            min_index = r
        if position != min_index:
            self.tasks[position], self.tasks[min_index] = self.tasks[min_index], self.tasks[position]
            self.shift_down(min_index)

    def next_priority(self):
        sorted_tasks = self.get_sorted_tasks()
        if len(sorted_tasks) > 1:
            return sorted_tasks[1]
        return Task()


    def get_sorted_tasks(self):


        temp = [t.copy() for t in self.tasks[:self.number_of_tasks]]

        def task_id(task):
            return f"{task.get_description()}||{task.get_activity()}"

        task_map = {task_id(task): task for task in temp}

        graph = defaultdict(list)
        reverse_deps = defaultdict(list)
        for task in temp:
            for dep in task.get_dependencies():
              for possible_dep in temp:
                if possible_dep.get_description() == dep:
                        graph[task_id(possible_dep)].append(task_id(task))
                        reverse_deps[task_id(task)].append(task_id(possible_dep))

        visited = {}
        result = []

        def dfs(node):
            if visited.get(node) == 'visiting':
                raise ValueError("Cycle detected in dependencies!")
            if visited.get(node) == 'visited':
                return
            visited[node] = 'visiting'
            for neighbor in graph[node]:
                dfs(neighbor)
            visited[node] = 'visited'
            result.append(node)

        for task in temp:
            tid = task_id(task)
            if visited.get(tid) is None:
                dfs(tid)

        sorted_tasks = [task_map[tid] for tid in reversed(result)]

        sorted_tasks = sorted(sorted_tasks, key=lambda t: (
           datetime.strptime(t.get_due_date(), "%Y-%m-%d"),
           t.get_priority()
        ))

        return sorted_tasks


    def print_tasks(self):
        sorted_tasks = self.get_sorted_tasks()
        print("------------------------------------------------------------")
        print("📋 TASKS SORTED BY: 1) Dependencies, 2) Due Date, 3) Priority")
        print("------------------------------------------------------------")

        print("\n✅ AVAILABLE TASKS (No dependencies or all dependencies completed):")
        available_tasks = [t for t in sorted_tasks
                            if all(dep not in [task.get_description() for task in sorted_tasks]
                                    for dep in t.get_dependencies())]

        if not available_tasks:
            print("🚫 No tasks available (all have unmet dependencies)")
        else:
            for i, t in enumerate(available_tasks, 1):
                print(f"\n🔹 Task {i}:")
                t.print_task()

        blocked_tasks = [t for t in sorted_tasks
                            if any(dep in [task.get_description() for task in sorted_tasks]
                                    for dep in t.get_dependencies())]

        if blocked_tasks:
            print("\n⛔ BLOCKED TASKS (Have unmet dependencies):")
            for i, t in enumerate(blocked_tasks, 1):
                print(f"\n🔸 Task {i}:")
                t.print_task()
                missing_deps = [dep for dep in t.get_dependencies()
                                    if dep in [task.get_description() for task in sorted_tasks]]
                print(f"  ❗ Missing dependencies: {', '.join(missing_deps)}")

        print("------------------------------------------------------------")

    def edit_task(self):
        if self.number_of_tasks == 0:
            print("🚫 No tasks to edit. 🤷")
            return

        self.print_tasks()
        keyword = input("🔍 Enter the description of the task you want to edit: ").strip()

        matching_tasks = [t for t in self.tasks if t.get_description().lower() == keyword.lower()]

        if not matching_tasks:
            print("❌ Task not found. 🔍")
            return
        elif len(matching_tasks) == 1:
            task_to_edit = matching_tasks[0]
        else:
            print("\n⚠ Multiple tasks found with the same description:")
            for i, t in enumerate(matching_tasks, 1):
                print(f"{i}. Activity: {t.get_activity()} | Priority: {t.get_priority()} | Work Date: {t.get_work_date()}")

            while True:
                try:
                    choice = int(input("Enter the number of the task you want to edit: "))
                    if 1 <= choice <= len(matching_tasks):
                        task_to_edit = matching_tasks[choice-1]
                        break
                    else:
                        print(f"❌ Please enter a number between 1 and {len(matching_tasks)}")
                except ValueError:
                    print("❌ Please enter a valid number.")

        print("\n✏ Editing task: ✍")
        task_to_edit.print_task()
        task_to_edit.input_task(self)
        print("✅ Task updated successfully. 🥳")

    def parent(self, position): return (position - 1) // 2
    def left_child_node(self, position): return 2 * position + 1
    def right_child_node(self, position): return 2 * position + 2

def main():
    name = input("👋 Enter your name: ")

    while True:
        max_tasks_input = input("🔢 Enter maximum number of tasks: ")
        if max_tasks_input.isdigit() and int(max_tasks_input) > 0:
            max_tasks = int(max_tasks_input)
            break
        else:
            print("❌ Invalid input! Please enter a valid positive integer for maximum number of tasks.")

    task_list = TaskList(max_tasks, name)
    task = Task()

    while True:
        print("-------------------------------------------------------------------------------")
        print("1️⃣ Press 1 to Add New Task ➕")
        print("2️⃣ Press 2 to Complete Highest Priority Task ✅")
        print("5️⃣ Press 3 to See Current Priority Task 🗓")
        print("4️⃣ Press 4 to See Next Priority Task ⏭")
        print("6️⃣ Press 5 to Display All Pending Tasks (Sorted by Dependencies, Due Date, Priority) 📋")
        print("7️⃣ Press 6 to Edit Existing Task ✏")
        print("8️⃣ Press 7 to Undo Last Action ↩")
        print("9️⃣ Press 8 to Exit 🚪")

        try:
            choice = int(input("➡ Enter your choice: "))
        except ValueError:
            print("❌ Invalid choice. Please enter a number between 1 and 8. 🤨")
            continue

        if choice == 1:
            if task_list.number_of_tasks == task_list.max_tasks:
                while True:
                    response = input("⚠ Task list is full. 😬\nWould you like to increase the task list limit? (yes/no): ").strip().lower()
                    if response == "yes":
                        try:
                            extra = int(input("➕ How many more tasks would you like to add? "))
                            if extra > 0:
                                task_list.max_tasks += extra
                                print(f"✅ Task limit increased. 👍")
                                break
                            else:
                                print("❌ Please enter a positive number.")
                        except ValueError:
                            print("❌ Please enter a valid number.")
                    elif response == "no":
                        print("🚫 No additional tasks added. 🤷")
                        break
                    else:
                        print("❌ Please answer with 'yes' or 'no'. 🤨")
                if task_list.number_of_tasks == task_list.max_tasks:
                    continue

            if task.input_task(task_list):
                task_list.add_task(task)
        elif choice == 2:
            task = task_list.remove_task()
            if task.get_priority() != 11:
                print("✅ Task Completed: 🥳")
                task.print_task()
        elif choice == 3:
            sorted_tasks = task_list.get_sorted_tasks()
            available_tasks = [t for t in sorted_tasks
                       if all(dep not in [task.get_description() for task in sorted_tasks]
                              for dep in t.get_dependencies())]
            if available_tasks:
                print("🗓 Current Scheduled Task (based on Final Sorted Tasks):")
                available_tasks[0].print_task()
            else:
                print("🚫 No tasks available for scheduling (unmet dependencies).")
        elif choice == 4:
            task = task_list.next_priority()
            if task.get_priority() != 11:
                print("⏭ Next Highest Priority Task: 🤔")
                task.print_task()
            else:
                print("⚠ Only one or no task available. 😕")
        elif choice == 5:
            task_list.print_tasks()
        elif choice == 6:
            task_list.edit_task()
        elif choice == 7:
            task_list.undo_last_action()
        elif choice == 8:
            print("🚪 Exiting program. Goodbye! 👋")
            break
        else:
            print("❌ Invalid choice. Please choose between 1 and 8. 🤨")

if __name__ == "__main__":
    main()

👋 Enter your name: Radhakrishna
🔢 Enter maximum number of tasks: 4
-------------------------------------------------------------------------------
1️⃣ Press 1 to Add New Task ➕
2️⃣ Press 2 to Complete Highest Priority Task ✅
5️⃣ Press 3 to See Current Priority Task 🗓
4️⃣ Press 4 to See Next Priority Task ⏭
6️⃣ Press 5 to Display All Pending Tasks (Sorted by Dependencies, Due Date, Priority) 📋
7️⃣ Press 6 to Edit Existing Task ✏
8️⃣ Press 7 to Undo Last Action ↩
9️⃣ Press 8 to Exit 🚪
➡ Enter your choice: 1
📝 Enter task description: dsa
🎯 Enter task activity: project

⏱ Choose time unit:
1. Hours ⏳
2. Minutes ⌛
Enter the number of the time unit (1 or 2.): 1
Enter time required in hours: 1
📅 Enter due date (YYYY-MM-DD): 2025-04-18
🎉 Due date is today! 🥳
⏳ Time left until midnight: 23 hour(s): 16 minute(s): 56 second(s).
📅 Enter work date (YYYY-MM-DD): 2025-04-18
⏰ Enter start time (HH:MM) in 24 hrs: 3:00
🏁 Enter end time (HH:MM) in 24 hrs: 4:00
⭐ Enter priority (1 = highest, 10 = lowest):